# Importando base de dados

In [1]:
import pandas as pd 
from numpy import random 
from random import randint 

In [2]:
nomes_masculinos = pd.read_json("https://servicodados.ibge.gov.br/api/v1/censos/nomes/ranking?qtd=20&sexo=m")
nomes_femininos = pd.read_json("https://servicodados.ibge.gov.br/api/v1/censos/nomes/ranking?qtd=20&sexo=f")
alunos = pd.concat([nomes_masculinos, nomes_femininos])

alunos['id'] = random.permutation(len(alunos))+1

alunos.reset_index(drop=True, inplace=True);

# Criando email 

In [3]:
dominio = ['@gmo.br', '@ortega.com.br']

In [4]:
alunos['dominio'] = random.choice(dominio, len(alunos))
alunos['email'] = (alunos.nome + alunos.dominio).str.lower()

# retornar esse ranking à tabela, ordenando pela quantidade de curso. 
del alunos['rank']
del alunos['regiao']

# Lendo os Cursos e Atribuindo a Quantidade


In [5]:
cursos = pd.read_html('http://tabela-cursos.herokuapp.com/index.html')[0]
cursos = cursos['Nome do curso'].to_list()

In [6]:
# sorteando a quantidade de cursos que cada aluno faz, para depois sortear quais serão os cursos
alunos['matriculas'] = [randint(1,5) for _ in range(len(alunos))]

#criando um array unico com a quantidade de cursos de cada aluno
quantidade_de_cursos_geral = []

for i in alunos['id']:
  quantidade = alunos[alunos['id']==i]['matriculas'].to_list()
  quantidade = quantidade[0]

  #Array que possui a quantidade de cursos seguindo a mesma disposição do DataFrame
  #Assim, é possível sortear a quantidad de cursos correto e atribui-los aos alunos
  quantidade_de_cursos_geral.append(quantidade)

# Quantidade de Alunos por Curso

In [7]:
quantidade_por_curso = dict(zip(cursos, [0 for _ in range(len(cursos))]))

def contador(curso: str) -> None:
  if curso in quantidade_por_curso:
    quantidade_por_curso[curso] += 1

#Designando os Cursos


In [8]:
# Criando uma lista contendo lista com os cursos dos alunos 
cursos_geral = []

for i in quantidade_de_cursos_geral:
  curso_aluno = set()

  while len(curso_aluno) < i:
    selecionado = random.choice(cursos)
    curso_aluno.add(selecionado)
    contador(selecionado) # função para contar a quantidade de vezes que o curso foi atribuido

  curso_aluno = list(curso_aluno)
  cursos_geral.append(curso_aluno) 

In [9]:
# Atribuindo a lista construida no bloco anterior
# contendo os cursos aos respectivos alunos

alunos['cursos'] = cursos_geral

#Organizando as Informações em DataFrames

In [10]:
alunos_por_curso = pd.DataFrame(list(quantidade_por_curso.items()), columns=['Nome', 'Quantidade'])
alunos_por_curso.sort_values(by='Quantidade', ascending=False, inplace=True)
alunos_por_curso.reset_index(drop=True, inplace=True)
alunos_por_curso

,Nome,Quantidade
0,Lógica de programação,9
1,Preparatório para certificação Java,9
2,Estatística básica,8
3,Cursos de Python,7
4,.NET para web,7
5,Internet das coisas,7
6,TDD com Java,7
7,Java para Web,7
8,Análise de dados,7
9,Desenvolvendo jogos,6


In [11]:
alunos[['nome', 'sexo', 'dominio', 'email', 'matriculas', 'cursos']]

,nome,sexo,dominio,email,matriculas,cursos
0,JOSE,M,@ortega.com.br,jose@ortega.com.br,4,"[Novas integrações com Java, Java para Web, Pr..."
1,JOAO,M,@ortega.com.br,joao@ortega.com.br,3,"[Desenvolvendo jogos, TDD com Java, Programaçã..."
2,ANTONIO,M,@ortega.com.br,antonio@ortega.com.br,1,[.NET para web]
3,FRANCISCO,M,@ortega.com.br,francisco@ortega.com.br,1,[Ruby on Rails]
4,CARLOS,M,@gmo.br,carlos@gmo.br,2,"[Persistência com .NET, Java para Web]"
5,PAULO,M,@gmo.br,paulo@gmo.br,5,"[Preparatório para certificação Java, Desenvol..."
6,PEDRO,M,@gmo.br,pedro@gmo.br,3,"[Hardware básico, Orientação objetos com Java,..."
7,LUCAS,M,@gmo.br,lucas@gmo.br,1,[Código limpo com C#]
8,LUIZ,M,@gmo.br,luiz@gmo.br,3,"[C# para Web, Novas integrações com Java, Orie..."
9,MARCOS,M,@gmo.br,marcos@gmo.br,2,"[Código limpo com C#, Estatística básica]"
